# ADVANCED DEEP LEARNING Assignment 1 (Group 4)

| SL.NO | BITS ID      | NAME                      | CONTRIBUTION |
|-------|-------------|---------------------------|--------------|
| 1     | 2024AA05733 | Bollam Manindra           | 100%          |
| 2     | 2024AA05734 | S.Hari Chandana           | 100%          |
| 3     | 2024AA05732 | Ashvini Satish Boshale    | 100%          |
| 4     | 2024AA05735 | Nikhil Yadav              | 100%          |
| 5     | 2024AA05946 | Deepthi Pullarkat         | 100%          |